In [1]:
from xml.etree import ElementTree
from xml.dom import minidom
import glob, os
from xml.etree.ElementTree import Element, SubElement, Comment
import subprocess as sp
import pandas as pd
from datetime import datetime
from osgeo import ogr

In [2]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

def writeOGRVRT(basename, fullpath):
    top = Element('OGRVRTDataSource')
    child = SubElement(top, 'OGRVRTLayer')
    child.set('name', basename)
    sub_child_1 = SubElement(child, 'SrcDataSource')
    sub_child_1.text = fullpath
    sub_child_2 = SubElement(child, 'GeometryType')
    sub_child_2.text = 'wkbPoint'
    sub_child_3 = SubElement(child, 'LayerSRS')
    sub_child_3.text = 'EPSG:28992'
    sub_child_4 = SubElement(child, 'GeometryField')
    sub_child_4.set('encoding','PointFromColumns')
    sub_child_4.set('x','field_1')
    sub_child_4.set('y','field_2')
    sub_child_4.set('z','field_3')

    return prettify(top)

In [3]:
ruweDataDir = r'D:\Projects\Pr\3317.20\BodempeilingScript\survey_ruweData'
workdir = r'D:\Projects\Pr\3317.20\BodempeilingScript\tmp'
bg_shp = r'D:\Projects\Pr\3317.20\BodempeilingScript//achtergrond_shape.shp'

driver = ogr.GetDriverByName('ESRI Shapefile')

dataSource = driver.Open(bg_shp, 0) # 0 means read-only. 1 means writeable.

# Check to see if shapefile is found.
if dataSource is None:
    print ('Could not open %s' % (bg_shp))
else:
    print ('Opened %s' % (bg_shp))
    layer = dataSource.GetLayer()
    bg_shp_features = layer.GetFeatureCount()
    print ('Name of layer: %s' % layer.GetDescription())
    print ("Number of features in %s: %d" % (os.path.basename(bg_shp),bg_shp_features))
    
    for i in range(bg_shp_features):
        feat = layer.GetFeature(i)
        obID = feat.GetField('OBJECTID')    
        print (obID)

Opened D:\Projects\Pr\3317.20\BodempeilingScript//achtergrond_shape.shp
Name of layer: achtergrond_shape
Number of features in achtergrond_shape.shp: 2
1
2


In [4]:
ogr2ogr = r'C:\Python35\Lib\site-packages\osgeo//ogr2ogr.exe'
gdalwarp = r'C:\Python35\Lib\site-packages\osgeo//gdalwarp.exe'
gdal_rasterize = r'C:\Python35\Lib\site-packages\osgeo//gdal_rasterize.exe'
gdal_translate = r'C:\Python35\Lib\site-packages\osgeo//gdal_translate.exe'
gdalbuildvrt = r'C:\Python35\Lib\site-packages\osgeo//gdalbuildvrt.exe'

In [7]:
for root, dirs, files in os.walk(ruweDataDir):
    for file in files:
        if file.endswith('.asc'):
            init_file = os.path.join( os.path.abspath(root), file )
            base, extension = os.path.splitext(file)
            print (init_file)
            
            # 1 convert survey data to CSV format
            asc2csv = os.path.join(workdir,base+'.csv')
            df = pd.read_csv(init_file, header=None)
            df.to_csv(asc2csv, header=False, index=False, sep=';')
            
            # 2 build OGRVRT from CSV file
            csv2ogrvrt = os.path.join(workdir,base+'.vrt')
            with open(csv2ogrvrt, 'w') as the_file:
                the_file.write(writeOGRVRT(base, asc2csv))
                
            # 3 create RASTER from OGRVRT
            ogrvrt2tif = os.path.join(workdir,base+'.tif')
            command = [gdal_rasterize, '-a','field_3','-tr','1.0','1.0','-l',base,csv2ogrvrt,ogrvrt2tif]
            print (sp.list2cmdline(command))
            norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
            norm.communicate()

            
            for feature in range(bg_shp_features):
                feat = layer.GetFeature(feature)
                obID = feat.GetField('OBJECTID')    
                print (obID)            
                
                # 4 clip point RASTER with feature achtergrond SHP
                tif2feat_point_tif = os.path.join(workdir,base+'_feature'+str(obID)+'_'+'.tif')
                command = [gdalwarp, '-srcnodata', '0', '-dstnodata', '-9999', '-overwrite', '-crop_to_cutline', 
                           '-cutline', bg_shp, '-cwhere', 'OBJECTID = '+str(obID), ogrvrt2tif, tif2feat_point_tif]            
                print (sp.list2cmdline(command))
                norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
                norm.communicate()
                
                # 5A convert feature achtergrond SHP to RASTER
                # issue is that it is converting the whole SHP to RASTER [with NoData values outside the selected object
                # instead of using some type of cutline function
                feat_bg_shp2feat_bg_tmp_tif = os.path.join(workdir,base+'_feature'+str(obID)+'_bg_tmp'+'.tif')
                command = [gdal_rasterize, '-a', 'NoDataValu', '-a_srs', 'EPSG:28992', '-where', 'OBJECTID = '+str(obID), 
                           '-tr', '1.0', '1.0', '-l', layer.GetDescription(), bg_shp, feat_bg_shp2feat_bg_tmp_tif]         
                print (sp.list2cmdline(command))
                norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
                norm.communicate()
                
                # 5B clip achtergrond RASTER
                feat_bg_tmp_tif2feat_bg_tif = os.path.join(workdir,base+'_feature'+str(obID)+'_bg'+'.tif')
                command = [gdalwarp, '-srcnodata', '-9999', '-dstnodata', '-9999', '-tr', '1.0', '1.0', '-overwrite', 
                           '-crop_to_cutline', '-cutline', bg_shp, '-cwhere', 'OBJECTID = '+str(obID), 
                           feat_bg_shp2feat_bg_tmp_tif, feat_bg_tmp_tif2feat_bg_tif]            
                print (sp.list2cmdline(command))
                norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
                norm.communicate()                                

                # if step 5 works, continue with testing step 6 and 7
                # 6 Build VRT data source of point RASTER and feature achtergrond RASTER                
                feat_bg_tif_feat_point_tif2feat_vrt = os.path.join(workdir,base+'_feature'+str(obID)+'_'+'.vrt')
                command = [gdalbuildvrt, '-srcnodata', '-9999', feat_bg_tif_feat_point_tif2feat_vrt, 
                           tif2feat_point_tif, feat_bg_tmp_tif2feat_bg_tif]
                print (sp.list2cmdline(command))
                norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
                norm.communicate()

                # 7 Convert VRT to ArcInfoASCII   
                print ('step 7:')
                feat_vrt2feat_asc = os.path.join(workdir,base+'_feature'+str(obID)+'_'+'.asc')
                command = [gdal_translate, '-of', 'AAIGrid', '-tr', '1.0', '1.0', feat_bg_tif_feat_point_tif2feat_vrt, 
                           feat_vrt2feat_asc]
                print (sp.list2cmdline(command))
                norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
                norm.communicate() 

D:\Projects\Pr\3317.20\BodempeilingScript\survey_ruweData\SUR-5081-ATU-a-150914-BAL-WR-MB-AJR-1x1.asc
C:\Python35\Lib\site-packages\osgeo//gdal_rasterize.exe -a field_3 -tr 1.0 1.0 -l SUR-5081-ATU-a-150914-BAL-WR-MB-AJR-1x1 D:\Projects\Pr\3317.20\BodempeilingScript\tmp\SUR-5081-ATU-a-150914-BAL-WR-MB-AJR-1x1.vrt D:\Projects\Pr\3317.20\BodempeilingScript\tmp\SUR-5081-ATU-a-150914-BAL-WR-MB-AJR-1x1.tif
1
C:\Python35\Lib\site-packages\osgeo//gdalwarp.exe -srcnodata 0 -dstnodata -9999 -overwrite -crop_to_cutline -cutline D:\Projects\Pr\3317.20\BodempeilingScript//achtergrond_shape.shp -cwhere "OBJECTID = 1" D:\Projects\Pr\3317.20\BodempeilingScript\tmp\SUR-5081-ATU-a-150914-BAL-WR-MB-AJR-1x1.tif D:\Projects\Pr\3317.20\BodempeilingScript\tmp\SUR-5081-ATU-a-150914-BAL-WR-MB-AJR-1x1_feature1_.tif
C:\Python35\Lib\site-packages\osgeo//gdal_rasterize.exe -a NoDataValu -a_srs EPSG:28992 -where "OBJECTID = 1" -tr 1.0 1.0 -l achtergrond_shape D:\Projects\Pr\3317.20\BodempeilingScript//achtergrond_s

In [ ]:
## GENERAL STEPS OUTLINING THE PROCEDURE
# 0 Convert survey data to CSV format
# df = pd.read_csv(old_name, header=None)
# df.to_csv(csv_out, header=False, index=False, sep=';')

# 1 Build OGR VRT data source from CSV
# with open(vrt_out, 'w') as the_file:
#     the_file.write(writeOGRVRT(base, csv_out))

# 2 Convert VRT to SHP ## maybe can skip this and convert OGR_VRT to RASTER
# command = [ogr2ogr, '-f','ESRI Shapefile',shp_out,vrt_out]

# 3 Convert point selection SHP to RASTER 
# command = [gdal_rasterize, '-a', 'field_3', '-tr', '1.0', '1.0', '-l', base, shp_out, tif_out]

# 4 Intersect point RASTER with achtergrond SHP  ## this should be replaced with an gdalwarp
# command = [ogr2ogr, '-clipsrclayer', polygonforclipping.shp, out.shp, in.shp]

# 5 Convert achtergrond SHP to RASTER
# command = [gdal_rasterize, '-a', 'field_empty', '-tr', '1.0', '1.0', '-l', base, shp_out, tif_out]

# 6 Build VRT data source of point RASTER and achtergrond RASTER
# command = [gdalbuildvrt, '-srcnodata', '0', 'out.vrt', pointRASTER.tif, achtergrondRASTER.tif] # order is not sure

# 7 Convert VRT to ASC
# command = [gdal_translate, '-a_nodata', '0', '-of', 'AAIGrid', in.vrt, out.asc]